<a href="https://colab.research.google.com/github/appletreeleaf/NLP_Projects/blob/main/%EB%82%98%EB%A7%8C%EC%9D%98_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 과제 2. 나만의 데이터 셋 만들기
기계 학습 분야에서 중요한 것 중 하나가 학습에 사용할 자료를 수집하고 관리하는 것입니다.
예를 들어 주어진 문장이 부정인지 긍정인지 판별하는 모델을 학습 시키기 위해서는 긍정 부정이 라벨링된 문장 자료가 필요합니다.

문장 | 긍부정
-------|--------
진짜 좋은 영화. 가히 올해의 영화라고 불릴만하다. | 긍정
이런 영화에 100억원 이상 때려 넣은 감독은 충무로에서 쫓겨나야한다. | 부정

그러나 우리가 이러한 자료를 직접 수집 및 라벨링하기는 쉽지 않습니다.

한편, 내부 연구나 교육적 목적으로 이미 가공된 인터넷의 글들을 수집하는 것은 공정이용으로 저작권법에 어긋나지 않습니다.
따라서 네이버 영화 리뷰를 크롤링하여 나만의 긍부정 문장 데이터 셋을 만들어 봅시다.


## 2-1. 네이버 영화 리뷰 크롤링 (40점)
현재 상영 중인 영화에 대한 영화 한 줄 평은 다음 링크에서 확인 가능합니다.

[https://movie.naver.com/movie/point/af/list.naver?&page=1](https://movie.naver.com/movie/point/af/list.naver?&page=1)

네이버 영화 한줄평의 주소는 다음과 같이 `<페이지 숫자>`에 해당되는 숫자를 넣으면 해당 페이지를 불러오는 것이 가능합니다.

https://movie.naver.com/movie/point/af/list.naver?&page= `<페이지 숫자>`

이러한 접근을 통해 영화 한 줄 평을 끍어모아 다음과 같은 형태의 `CSV` 파일로 저장하는 코드를 만들어 봅시다.

movie | sentence | score
------|------|------
모가디슈 | 재미잇게 잘봤습니다. 배우들 연기에 더몰입감있게 봤습니다. | 10
낙원의 밤 | 차승원의 유머만 남은 흉내 낸 누아르, 결말조차 완벽하게 폭망 | 2
국화꽃 향기 | 뭔가 와닿지 않는 감동 그리고 스토리. 아쉽지만 여운이 남지 않는다 | 4
미쓰 홍당무 | 서우가 아주 매력적으로 나오는 영화 | 8
더 수어사이드 스쿼드 | 후련하게 봤어요 재밌네요ㅋㅋㅋ잔인하고 징그러운거 좋아하시는 분들께 추천 | 9

### 크롤러를 만들때 다음과 같은 사항을 준수해야 합니다.
1. 실제 학습을 진행할 것은 아니기 때문에 문장은 1000개 정도만 가져오면 충분합니다.
2. 각 페이지를 조회하실때 시간차가 없다면 DDOS 공격으로 분류되어 페이지 접속이 막힐 수가 있습니다. 페이지 조회시 최소 0.5초의 시간 차를 둡시다.
3. 신고되어 삭제된 한줄평은 데이터에 들어가면 안됩니다. 또한 한줄평이 아예 없는 경우도 제외하셔야 합니다.
4. 사용 가능한 라이브러리는 다음과 같습니다.
    * 파이썬 표준 라이브러리
    * Request
    * Beutiful Soup
    * (For colab user) Colab
    * (Optional) Scrapy
5. `CSV` 파일로 저장하실때 Unicode로 저장하셔야 하며, Excel에서 유니코드 형식으로 열었을 때 문제 없이 열 수 있어야 합니다. (excel의 인코딩 기본값은 EUC-KR이므로 그냥 열게되면 인코딩이 깨질 수 있습니다.)
6. `CSV` 파일은 쉼표로 구분되어야 합니다.
7. `CSV` 파일의 레코드 형식은 다음과 같습니다.
  * movie - 영화 이름이 들어갑니다. 문자열입니다.
  * sentence - 한줄평이 들어갑니다. 문자열입니다.
  * score - 별점이 들어갑니다. 숫자입니다.
8. 크롤링 코드를 실행하여 크롤링을 한 번에 완료해야 합니다.
9. 그외 상세한 구현 방법은 자율입니다.

### 과제를 제출하실때 다음을 제출해야합니다.
1. 크롤링된 데이터를 저장한 `samples.csv`
2. 크롤링 코드


# 라이브러리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv

need_reviews_cnt = 1000
reviews = []
review_data=[]

#page를 1부터 1씩 증가하며 URL을 다음 페이지로 바꿈
for page in range(1,200):
    url = f'https://movie.naver.com/movie/point/af/list.naver?&page={page}'
    #get : request로 url의  html문서의 내용 요청
    html = requests.get(url)
    #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
    soup = BeautifulSoup(html.content,'html.parser')
    #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
    reviews = soup.find_all("td",{"class":"title"})

    #한 페이지의 리뷰 리스트의 리뷰를 하나씩 보면서 데이터 추출
    for review in reviews:
        sentence = review.find("a",{"class":"report"}).get("onclick").split("', '")[2]
        #만약 리뷰 내용이 비어있다면 데이터를 사용하지 않음
        if sentence != "":
            movie = review.find("a",{"class":"movie color_b"}).get_text()
            score = review.find("em").get_text()
            review_data.append([movie,sentence,int(score)])
            need_reviews_cnt-= 1
    #현재까지 수집된 리뷰가 목표 수집 리뷰보다 많아진 경우 크롤링 중지
    if need_reviews_cnt < 0:
        break
    #다음 페이지를 조회하기 전 0.5초 시간 차를 두기
    time.sleep(0.5)

In [ ]:
columns_name = ["movie","sentence","score"]
with open( "samples.csv", "w", newline ="",encoding = 'utf8' ) as f:
    write = csv.writer(f)
    write.writerow(columns_name)
    write.writerows(review_data)
    print(review_data)

[]


In [ ]:
import csv
with open('/content/drive/MyDrive/Colab Notebooks/samples.csv', 'r') as fd:
  reader = csv.reader(fd,
      delimiter=',',
      quotechar='"',
      quoting=csv.QUOTE_MINIMAL
      )
  for entry in reader:
    print(entry)

['\ufeffmovie', 'sentence', 'score']
['영웅', '사실적 스토리 전개에 연기력이 뛰어나끝날때까지 몰입도가 최고였다^^', '10']
['박서방', '크리스마스에 티비에서 하길래 봤어요...그 옛날 서울 분위기도 아버지의 사랑도 너무 감동ㅜㅠ', '10']
['오늘 밤, 세계에서 이 사랑이 사라진다 해도', '와....슌스케개잘생김', '10']
['아바타: 물의 길', '전편과 같은 레퍼토리였고 스토리에 대한 좋지 못한 후기들을 접하고 기대없이 보러갔음뜻밖에 가족적인 서사에 눈물을 많이 흘려 전편과 같은 대결적 구도의 스토리가 커버됐음영상미는 당연히 말할 것도 없이 몰입해서 보았음영상미를 극대화해서 봐야 할 영화이기 때문에 아이멕스 스리디로 봐야할 영화', '10']
['20세기 소녀', '뻔한 스토리지만 좋은 연출과 연기 덕분에 재밌게 감상했다. 각 배역들의 캐릭터 설정이 잘되있어서 더 몰입하며 봤다', '10']
['아바타: 물의 길', '3시간이라 오줌참기 개힘든거 빼곤 너무 재밋게봄', '9']
['영웅', '너무 슬프다 압도적인 김고은', '10']
['영웅', '당연했던 것에 감사함을 느끼게 해주는 영화', '10']
['오늘 밤, 세계에서 이 사랑이 사라진다 해도', '너무 재밌었어요ㅠㅠ소설을 먼저 봐서 내용을 알고 있기에재미없을까봐 걱정했는데오히려 영화 속 장면이 소설에서 어떻게 표현 됐었나 대비하며 보게 되어서 더 재밌었어요!!!', '10']
['머니볼', '영화의 가장 큰 장점 중 하나는 우리가 평소에 경험하기 힘든 분야도 간접적으로 체험할 수 있게 해준다는 것이다. 스포츠 팬이라면 누구나 흥미롭게 볼 수 있는 영화', '8']
['버드맨', '예전에 보다가 재미 없어서 쟁여놓다 이제야 다시 봤는데 두고 첫 장면 부터 꽃가게에서 동양인 한테 입 닥치라고 하면서 가게 안에서 역겨운 김치 냄새 난다로 뜬금 없는 비하로 시작함 그 뒤로 대부분이 연극 내용이고 포르노 연기가 섞여 있으니 청소년은 시청 못하게 하시고요 

In [ ]:
print(reader)

TypeError: ignored

## 2-2. 네이버 영화 데이터 셋 제작 (30점)
`CSV` 파일로 저장된 데이터를 쉽게 접근하기 위해서는 파이썬 내에서 구조화할 필요가 있습니다.
이를 위하여 데이터에 접근 가능한 `RawMovieReview` 클래스를 만들어 봅시다.


### `RawMovieReview` 클래스는 다음을 준수해야 합니다.
0. 이하 모든 클래스 공통 사항
    * 클래스 속성은 사용이 금지됩니다.
    * 전역 변수 사용이 금지됩니다.
1. 생성자
    * 생성자의 추가 인자는 `str`타입의 `file_name` 하나만을 받습니다.
    * `file_name` 인자의 기본값은 `"samples.csv"`입니다.
    * 해당 객체는 해당 csv 파일을 다루는 객체가 되어야 합니다.
    * 모든 속성 값은 `protected` 홋은 `private`이어야 합니다. 즉, 메소드만 `public`입니다.
2. Indexing
    * 대괄호로 N번째 sample에 접근할 수 있어야 합니다.
    * N은 0부터 시작합니다. 즉, `dataset[0]`은 첫번째 한줄 평 입니다.
    * Indexing 결과값은 `영화 이름, 한줄평, 점수`로 타입은 `(str, str, int)` 형태의 튜플입니다.
    * 대괄호로 읽기만 가능하고 수정은 불가능해야 합니다.
3. Length
    * `len(dataset)`의 형태로 데이터셋 내의 한줄평 개수를 조회할 수 있어야 합니다.

### 과제를 제출하실때 다음을 제출해야합니다.
1. `RawMoviewReview` 클래스가 정의된 코드 (`.py` 혹은 `.ipynb`)


In [ ]:
import csv

with open('samples.csv', 'w') as fd:
    writer = csv.writer(fd)
    writer.writerow(['movie', 'sentence', 'score'])
    writer.writerows(samples)

In [ ]:
class RawMovieReview:
    def __init__(self, file_name: str="samples.csv"):
        with open(file_name, 'r') as fd:
            fd.readline()
            reader = csv.reader(fd)
            self._samples = [
                (movie, sentence, int(score))
                for movie, sentence, score
                in reader
            ]

    def __len__(self):
        return len(self._samples)

    def __getitem__(self, index):
        return self._samples[index]

dataset = RawMovieReview()
print(len(dataset))
print(dataset[321])


## 2-3. 네이버 영화 학습 데이터 셋 제작 (30점)
위에서 제작한 데이터 셋은 `CSV` 파일을 모두 조회하지만, 학습에 바로 사용되기에는 다소 불편합니다.
따라서 이를 상속한 `MovieReview`를 만들어 봅시다.

### `MovieReview` 클래스는 다음을 준수해야 합니다.
1. 상속
    * 위에서 정의한 `RawMoviewReview` 클래스를 상속받아야합니다.
    * `RawMoviewReview` 내의 데이터를 복사하면 안됩니다.
    * 즉, 저장된 `CSV`파일은 부모 클래스의 속성 혹은 메소드로 접근해야합니다.
2. 생성자
    * 생성자의 인자는 부모의 인자와 `int`타입의 `score_threadhold`를 받습니다.
    * 해당 자식 클래스에는 `CSV`파일 및 그 내용이 속성으로 들어가면 안됩니다.
    * 즉, 영화 데이터는 부모에만 저장되어야합니다.
3. Indexing
    * 부모의 Indexing을 재정의합니다. (Overriding)
    * 기본적인 기능은 부모의 기능과 비슷하나 이하의 사항이 다릅니다.
    * Indexing 결과 값은 `한줄평, 긍부정`으로 타입은 `(str, bool)` 형태의 튜플입니다.
    * 점수가 객체의 `score_threadhold` 이상일 경우 긍정이 `True`, 미만이면 `False` 입니다.
    * 한줄평 string의 경우 양쪽에 공백이 없어야 합니다.

### 과제를 제출하실때 다음을 제출해야합니다.
1. `MoviewReview` 클래스가 정의된 코드 (`.py` 혹은 `.ipynb`)

In [ ]:
class MovieReview(RawMovieReview):
    def __init__(self, score_threadhold: int, file_name: str="samples.csv"):
        super().__init__(file_name)
        self._score_threadhold = score_threadhold

    def __getitem__(self, index):
        _, sentence, score = super().__getitem__(index)
        return sentence, score >= self._score_threadhold

dataset = MovieReview(7)
print(len(dataset))
print(dataset[321])

## 최종 제출
* 모든 파일을 `zip`으로 압축하여 `HW2_<한글 이름>.zip` 형태로 제출합니다.


## 채점 기준 표
각각의 문제에 대해서 이하의 감점이 각각 들어갈 수 있습니다. 음수 점수는 없습니다.
아래의 채점 기준표는 확정이 아니며, 추후 수정된 점수 기준이 적용될 수 있습니다.

감점요소 | 설명 | 감점 점수
-----|-----|-----
API 문서와 다른 입출력 | 과제 내 명시된 API 및 포멧과 입출력이 다릅니다. | -15
비효율적인 시간복잡도 | 시간복잡도가 최선이 아닙니다. (크롤링 $O(N)$, Indexing $O(1)$)| -10
비효율적인 공간 구조 | 부모 자식간 데이터가 중복하는 형태로 클래스가 작성되어 있습니다. | -10
명시되어 있지 않은 외부 라이브러리 사용 | 지정된 라이브러리 외에 다른 라이브러리를 사용하였습니다 | -10
권장하지 않는 문법 | 전역 변수나 클래스 속성을 사용하였습니다. | -10